In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.model_selection import train_test_split

df = pd.read_csv('df_final.csv')

files  = df['img'].values
labels = df['has_cancer'].values.astype(bool)

train_files, test_files, y_train, y_test = train_test_split(files, labels, test_size=0.33, random_state=42)

class CancerDataset(Dataset):
    def __init__(self, file_paths, targets, transform=None):
        self.file_paths = [f"data_raw/{file_path}" for file_path in file_paths]
        self.targets = targets
        self.transform = transform
    
    # necesitamos saber la longitud para luego saber cuantos batches vamos a necesitar
    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, index):
        image = Image.open(self.file_paths[index]).convert('L')
        if self.transform:
            image = self.transform(image)
        targets = self.targets[index]

        return image, targets


transform = transforms.Compose([
    transforms.Resize((256, 256)),  # reducimos tamaño
    transforms.ToTensor(),          # convierte a tensor y normaliza a [0,1]
])
    # transforms.ToTensor() ya hace esto
# Con unsqueeze le añado una dimensión en la posicion 0 con un 1, que es la dimensión del color donde le decimos al modelo que solo hay uno, es decir gris
# dividimos los pixeles entre 255 para tener valores noramlizados entre 0 y 1
train_dataset = CancerDataset(train_files, y_train, transform=transform)
test_dataset = CancerDataset(test_files, y_test, transform=transform)

Hay que pasar todo a tensores para que el modelo pueda ser entrenado

In [2]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_dataset,  batch_size=64, shuffle=False, num_workers=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1    = nn.Conv2d(1, 32, 3, padding=1)
        self.bn1      = nn.BatchNorm2d(32)
        self.conv2    = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2      = nn.BatchNorm2d(64)
        self.pool     = nn.MaxPool2d(2, 2)
        self.adapt    = nn.AdaptiveAvgPool2d((8, 8))
        self.fc1      = nn.Linear(64 * 8 * 8, 256)
        self.fc2      = nn.Linear(256, 128)
        self.fc3      = nn.Linear(128, 1)
        self.dropout  = nn.Dropout(0.5)
        self.relu     = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.adapt(x)
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x.squeeze(1)

model = CNN().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

num_epochs  = 10

for epoch in range(1, num_epochs+1):
    # — Training —
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)                     # [B,1,256,256]
        labels = labels.float().to(device)             # BCEWithLogits expects float
        
        optimizer.zero_grad()
        logits = model(images).view(-1)                # [B]
        loss   = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    
    # — Validation —
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            logits = model(images).view(-1)
            preds  = (torch.sigmoid(logits) > 0.5).long()
            correct += (preds == labels.long()).sum().item()
            total   += labels.size(0)
    accuracy = correct / total * 100
    
    print(f"Epoch {epoch:2d} — Loss: {epoch_loss:.4f} — Val Acc: {accuracy:.2f}%")


Epoch  1 — Loss: 0.3140 — Val Acc: 93.31%
Epoch  2 — Loss: 0.2022 — Val Acc: 95.57%
Epoch  3 — Loss: 0.1480 — Val Acc: 96.53%
Epoch  4 — Loss: 0.1328 — Val Acc: 96.61%
Epoch  5 — Loss: 0.1151 — Val Acc: 96.78%
Epoch  6 — Loss: 0.1089 — Val Acc: 96.38%
Epoch  7 — Loss: 0.0964 — Val Acc: 96.06%
Epoch  8 — Loss: 0.0937 — Val Acc: 97.44%
Epoch  9 — Loss: 0.0893 — Val Acc: 97.29%
Epoch 10 — Loss: 0.0858 — Val Acc: 97.46%


Guardo los pesos, luego para llamarlo desde fuera tengo que nombrar al modelo 

model = CNN().to(device)

state_dict = torch.load("reconocimientoCancer.pth", map_location=device)

model.load_state_dict(state_dict)

model.eval()  

In [ ]:
torch.save(model.state_dict(), "./streamlit/reconocimientoCancer.pth")